In [1]:
import os
import itertools
import numpy as np
from collections import Counter
from fractions import Fraction
from math import comb
from math import sqrt

## 51 - Prime Families

In [2]:
def primes_by_digit(ss):
    prime_array = np.ones(ss)
    prime_array[0] = 0
    prime_array[1] = 0
    primes_by_digit_count =[]
    curr_list = []
    curr_index = 1
    for i in range(2,ss):
        if prime_array[i]:
            index = int(np.floor(np.log10(i)))
            if index == curr_index:
                curr_list.append(str(i))
            else:
                curr_index = index
                primes_by_digit_count.append(curr_list)
                curr_list = []
                curr_list.append(str(i))
            for j in range(i*2, ss, i):
                prime_array[j] = 0
    return primes_by_digit_count

In [3]:
def num_filter(num, pattern):
    new_num = ""
    char_c = None
    for c1, c2 in zip(num, pattern):
        if c2 == '1':
            new_num+=c1
        else:
            if not char_c:
                char_c = c1
            elif char_c != c1:
                return None
            new_num+='*'
    return new_num

In [4]:
def build_pattern(pattern):
    possibles = []
    for i in range(10):
        this_num = ""
        for char in pattern:
            if char == '*':
                this_num += str(i)
            else:
                this_num += char
        possibles.append(this_num)
    return possibles

In [5]:
%%time
def pe_51():
    prime_array = primes_by_digit(10**7 + 20)
    for n,prime_list in enumerate(prime_array):
        for k in range(1, n-1):
            for ones_pos in itertools.combinations(range(n), k):
                bits = ['0'] * n
                for pos in ones_pos:
                    bits[pos] = '1'
                combo = ''.join(bits)
                filtered_list = []
                for prime in prime_list:
                    new_num = num_filter(prime, combo)
                    if new_num:
                        filtered_list.append(new_num)
                pattern_dict = dict(Counter(filtered_list))
                if max(pattern_dict.values()) == 8:
                    pattern = max(pattern_dict, key=pattern_dict.get)
                    print(pattern)
                    possibles = build_pattern(pattern)
                    for possible in possibles:
                        if possible in prime_list:
                            print(possible)
                            return
pe_51()

*2*3*3
121313
CPU times: user 4.09 s, sys: 43.2 ms, total: 4.13 s
Wall time: 4.13 s


# 52 - Permuted Multiples

In [6]:
def digits(n):
    digit_list = []
    for char in str(n):
        digit_list.append(char)
    digit_list.sort()
    return ''.join(digit_list)

In [7]:
%%time
def pe_52():
    for i in range(2, 10**6):
        multiple_digit_list = []
        for j in range(1, 7):
            multiple_digit_list.append(digits(i*j))
        if len(np.unique(multiple_digit_list)) == 1:
            print(i)
            return
pe_52()

142857
CPU times: user 911 ms, sys: 0 ns, total: 911 ms
Wall time: 910 ms


# 53 - Large Combinatorics

In [8]:
%%time
def pe_53():
    total = 0
    for n in range(1, 101):
        for r in range(1, n+1):
            if comb(n, r) > 10**6:
                total += 1
    print(total)
pe_53()

4075
CPU times: user 400 μs, sys: 0 ns, total: 400 μs
Wall time: 407 μs


# 54 - Poker Hands

In [9]:
def judge_hand(hand):
    hand = hand.split()
    player_one_rank, player_one_cards = get_rank(hand[:5])
    player_two_rank, player_two_cards = get_rank(hand[5:])
    if player_one_rank>player_two_rank:
        return 1
    elif player_one_rank == player_two_rank:
        for p1, p2 in zip(player_one_cards, player_two_cards):
            if p1 > p2:
                return 1
            elif p2 > p1:
                return 0
            else:
                continue
    else:
        return 0

In [10]:
def get_rank(cards):
    face_card_map = {'T':10, 'J':11, 'Q':12, 'K':13, 'A':14}
    values = []
    suits = []
    for card in cards:
        suits.append(card[-1])
        if card[:-1] in face_card_map:
            values.append(face_card_map[card[:-1]])
        else:
            values.append(int(card[:-1]))
    values = np.sort(values)
    values = values[::-1]
    unique_values, counts = np.unique(values, return_counts=True)
    num_unique_cards = len(unique_values)

    if num_unique_cards == 5:
        flush = True if len(np.unique(suits))==1 else False
        straight = True
        for i in range(0, 4):
            if values[i]-1 != values[i+1]:
                straight = False
                break
        if np.array_equal(values , [14, 5, 4, 3, 2]):
            straight = True
            values = [5, 4, 3, 2, 1]
            
        if straight and flush:    
            return (8, values)       
        elif flush:
            return (5, values)
        elif straight:
            return (4, values)
        else:
            return (0, values)
    else:
        sorted_pairs = sorted(zip(unique_values, counts), key=lambda x: (-x[1], -x[0]))
        sorted_values, sorted_counts = zip(*sorted_pairs)
        
        if num_unique_cards == 4:
            return (1, sorted_values)
        elif num_unique_cards == 3:
            if sorted_counts[0] == 2:
                return (2, sorted_values)
            else:
                return (3, sorted_values)
        elif num_unique_cards == 2:
            if sorted_counts[0] == 3:
                return (6, sorted_values)
            else:
                return (7, sorted_values)

In [11]:
%%time
def pe_54():
    path = "./inputs/0054_poker.txt"
    if os.path.exists(path):
        with open(path, 'r') as file:
            hands = [line.strip() for line in file]
            print(sum([judge_hand(hand) for hand in hands]))
pe_54()

376
CPU times: user 48.7 ms, sys: 4.12 ms, total: 52.8 ms
Wall time: 41.2 ms


# 55 - Lychrel Numbers

In [12]:
def reverse_num(num_int):
    num_str = str(num_int)
    num_str = num_str[::-1]
    return int(num_str)

In [13]:
def is_palindrome(num_int):
    num_str = str(num_int)
    str_len = len(num_str)
    for i in range(str_len // 2):
        if num_str[i] != num_str[str_len-i-1]: return False
    return True    

In [14]:
%%time
def pe_55():
    lychrel_nums = 0
    for n in range(1, 10**4):
        is_lychrel = True
        num = n
        for i in range(50):
            num += reverse_num(num)
            if is_palindrome(num):
                is_lychrel = False
                break
        if is_lychrel: lychrel_nums += 1
    print(lychrel_nums)
pe_55()

249
CPU times: user 22.2 ms, sys: 47 μs, total: 22.2 ms
Wall time: 21.7 ms


# 56 - Powerful Digit Sums

In [15]:
%%time
def pe_56():
    max_sum = 0
    for a in range(1, 100):
        for b in range(1, 100):
            curr_sum = sum([int(char) for char in str(a**b)])
            if curr_sum > max_sum: max_sum = curr_sum
    print(max_sum)
pe_56()

972
CPU times: user 60.9 ms, sys: 13 μs, total: 60.9 ms
Wall time: 60.4 ms


# 57 - Square Root Convergents

In [16]:
%%time
def pe_57():
    frac = Fraction(1, 2)
    total = 0
    for n in range(10**3):
        frac_n = frac + 1
        if len(str(frac_n.numerator)) > len(str(frac_n.denominator)): total += 1
        frac = frac + 2
        frac = 1 / frac
    print(total)
pe_57()

153
CPU times: user 4.64 ms, sys: 0 ns, total: 4.64 ms
Wall time: 4.52 ms


# 58 - Number Spiral Diagonals

In [17]:
def prime_array(size):
    prime_array = np.ones(size, np.bool_)
    prime_array[0] = False
    prime_array[1] = False
    sqrt_size = int(sqrt(size)) + 1
    for i in range(2,sqrt_size):
        if prime_array[i]:
            prime_array[i*i:size:i] = False
    return prime_array

In [18]:
%%time
def pe_58():
    primes = prime_array(10**9)
    num = 1
    diff = 0
    total_primes = 0
    total = 1
    for ring in range(1,10**10):
        diff += 2
        for side in range(4):
            num += diff
            total +=1
            if primes[num]: total_primes += 1
        if total_primes/total < 0.1:
            print(ring*2 + 1)
            return
pe_58()

26241
CPU times: user 5.93 s, sys: 75.8 ms, total: 6.01 s
Wall time: 6.01 s


# 59 - XOR Decryption

In [19]:
%%time
def pe_59():
    path = "./inputs/0059_cipher.txt"
    if os.path.exists(path):
        with open(path, 'r') as file:
            cipher_ascii = [int(item.strip()) for item in file.read().split(",")]
    key_len = int(len(cipher_ascii) / 3)
    common_words = ['and', 'the', 'for', 'of']
    for a in range(97, 123):
        for b in range(97, 123):
            for c in range(97, 123):
                passwd = [a, b, c]
                key = []
                for _ in range(key_len):
                    key.extend(passwd)
                plaintext_vals = [j^k for j,k in zip(key, cipher_ascii)]
                plaintext_chars = [chr(val) for val in plaintext_vals]
                plaintext = ''.join(plaintext_chars)
                if all(word in plaintext.lower() for word in common_words):
                    print(plaintext)
                    print(sum(plaintext_vals))
                    return
pe_59()

An extract taken from the introduction of one of Euler's most celebrated papers, "De summis serierum reciprocarum" [On the sums of series of reciprocals]: I have recently found, quite unexpectedly, an elegant expression for the entire sum of this series 1 + 1/4 + 1/9 + 1/16 + etc., which depends on the quadrature of the circle, so that if the true sum of this series is obtained, from it at once the quadrature of the circle follows. Namely, I have found that the sum of this series is a sixth part of the square of the perimeter of the circle whose diameter is 1; or by putting the sum of this series equal to s, it has the ratio sqrt(6) multiplied by s to 1 of the perimeter to the diameter. I will soon show that the sum of this series to be approximately 1.644934066842264364; and from multiplying this number by six, and then taking the square root, the number 3.141592653589793238 is indeed produced, which expresses the perimeter of a circle whose diameter is 1. Following again the same ste

# 60 - Prime Pair Sets

In [20]:
def prime_array(size):
    prime_array = np.ones(size, np.bool_)
    prime_array[0] = False
    prime_array[1] = False
    sqrt_size = int(sqrt(size)) + 1
    for i in range(2,sqrt_size):
        if prime_array[i]:
            prime_array[i*i:size:i] = False
    return prime_array

In [21]:
def unique_list(lst):
    seen = set()
    result = []
    for item in lst:
        t = tuple(item)
        if t not in seen:
            seen.add(t)
            result.append(item)
    return result

In [22]:
def is_valid_family(family, num, primes):
    for j in range(len(family)):
        if not primes[int(str(family[j]) + str(num))] or not primes[int(str(num) + str(family[j]))]: return False
    return True

In [23]:
%%time
def pe_60():
    primes = prime_array(10**10)
    family_list = []
    for n in range(10**9):
        if not n%20:
            for k in range(len(family_list)):
                family_list[k] = sorted(family_list[k])
            family_list = unique_list(family_list)
        
        if not primes[n]: continue
        n_str = str(n)
        n_str_len = len(str(n))
        for i in range(1, n_str_len):
            s1 = int(n_str[:i])
            s2 = int(n_str[i:])
            flip = int(str(s2) + str(s1))
            if len(str(flip)) != n_str_len: continue
            if primes[s1] and primes[s2] and primes[flip]:
                for family in family_list:
                    if s1 in family and not s2 in family and is_valid_family(family, s2, primes):
                        new_family = family + [s2]
                        if len(new_family) ==5:
                            print(f"Family: {new_family}\tSum: {sum(new_family)}")
                            return
                        family_list.append(new_family)
                    if s2 in family and not s1 in family and is_valid_family(family, s1, primes):
                        new_family = family + [s1]
                        if len(new_family) ==5:
                            print(f"Family: {new_family}\tSum: {sum(new_family)}")
                            return
                        family_list.append(new_family)
                family_list.append([s1, s2])
                
                for k in range(len(family_list)):
                    family_list[k] = sorted(family_list[k])
                family_list = unique_list(family_list)
pe_60()

Family: [13, 5197, 5701, 6733, 8389]	Sum: 26033
CPU times: user 1min 7s, sys: 1.26 s, total: 1min 9s
Wall time: 1min 9s
